In [6]:
filepath = "file://///home/ubuntu/Downloads/SparkBasic/NYPD_7_Major_Felony_Incidents.csv"
NYPD = sc.textFile(filepath)

header = NYPD.first()

dataWOHeader = NYPD.filter(lambda i: i != header)

import csv
from io import StringIO
from collections import namedtuple

fields = header.replace(" ","_").replace("/","_").split(",")

Crime = namedtuple('Crime', fields, verbose=True)

def parse(row):
    reader = csv.reader(StringIO(row))
    row=next(reader)
    return Crime(*row)

crimes = dataWOHeader.map(parse)

crimes.map(lambda i: i.Offense).countByValue()

crimes.map(lambda i:i.Occurrence_Year).countByValue()

crimesFiltered = crimes.filter(lambda i: not (i.Offense=="NA" or i.Occurrence_Year==''))\
                    .filter(lambda i: int(i.Occurrence_Year)>=2006)

from builtins import property as _property, tuple as _tuple
from operator import itemgetter as _itemgetter
from collections import OrderedDict

class Crime(tuple):
    'Crime(OBJECTID, Identifier, Occurrence_Date, Day_of_Week, Occurrence_Month, Occurrence_Day, Occurrence_Year, Occurrence_Hour, CompStat_Month, CompStat_Day, CompStat_Year, Offense, Offense_Classification, Sector, Precinct, Borough, Jurisdiction, XCoordinate, YCoordinate, Location_1)'

    __slots__ = ()

    _fields = ('OBJECTID', 'Identifier', 'Occurrence_Date', 'Day_of_Week', 'Occurrence_Month', 'Occurrence_Day', 'Occurrence_Year', 'Occurrence_Hour', 'CompStat_Month', 'CompStat_Day', 'CompStat_Year', 'Offense', 'Offense_Classification', 'Sector', 'Precinct', 'Borough', 'Jurisdiction', 'XCoordinate', 'YCoordinate', 'Location_1')

    def __new__(_cls, OBJECTID, Identifier, Occurrence_Date, Day_of_Week, Occurrence_Month, Occurrence_Day, Occurrence_Year, Occurrence_Hour, CompStat_Month, CompStat_Day, CompStat_Year, Offens

In [3]:
def extractCoordi(location):
    location_lat = float(location[1:location.index(",")])
    location_lon = float(location[location.index(",")+1:-1])
    return (location_lat,location_lon)

In [7]:
crimesFiltered.map(lambda i: extractCoordi(i.Location_1))

PythonRDD[12] at RDD at PythonRDD.scala:48

# Use Reduce

In [11]:
crimesFiltered.map(lambda i: extractCoordi(i.Location_1))\
                .reduce(lambda i,j:(min(i[0],j[0]),min(i[1],j[1])))

(40.112709974, -77.519206334)

In [12]:
crimesFiltered.map(lambda i: extractCoordi(i.Location_1))\
                .reduce(lambda i,j:(max(i[0],j[0]),max(i[1],j[1])))

(59.5805088160001, -73.700716685)

In [13]:
#check boundries of US on Google Map and use filter to filter out useless data

In [14]:
crimesFinal = crimesFiltered.filter(lambda x:extractCoordi(x.Location_1)[0]>=40.47 and\
                                    extractCoordi(x.Location_1)[0]<=40.91 and \
                                   extractCoordi(x.Location_1)[1]>= -74.25 and \
                                   extractCoordi(x.Location_1)[1]<= -73.70)

In [15]:
#TREND BY YEAR

crimesFinal.map(lambda i:i.Occurrence_Year).countByValue()

defaultdict(int,
            {'2006': 127825,
             '2007': 120170,
             '2008': 117329,
             '2009': 105974,
             '2010': 105600,
             '2011': 107170,
             '2012': 111751,
             '2013': 111247,
             '2014': 106825,
             '2015': 102627})

# Summarizing and Visualizing Crime in NYC

In [19]:
crimesFinal.filter(lambda i : i.Offense == 'BURGLARY')\
                    .map(lambda i:i.Occurrence_Year)\
                    .countByValue()

defaultdict(int,
            {'2006': 23055,
             '2007': 21652,
             '2008': 20726,
             '2009': 19430,
             '2010': 18686,
             '2011': 18852,
             '2012': 19303,
             '2013': 17410,
             '2014': 16827,
             '2015': 14958})